# ORM Basics
This notebook covers the following:

- Model (basic fields)
- Add
- Select
- Update
- Delete

The syntax used here is geared towards SQLAlchemy 2.0, but is likely compatible with most of 1.4.
Any 1.4 specific edge cases should be noted.

# Setup

In [1]:
import sqlalchemy as sa
from sqlalchemy import orm
print('SQLAlchemy Version:', sa.__version__)

SQLAlchemy Version: 2.0.29


In [2]:
from contextlib import contextmanager
import logging
import sys

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARN)

logger = logging.getLogger('sqlalchemy.engine')
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

@contextmanager
def logs(level=logging.INFO):
    state = handler.level
    handler.setLevel(level)
    try:
        yield
    finally:
        handler.setLevel(state)

In [3]:
engine = sa.create_engine('sqlite://')
con = engine.connect()

# Model
Defining a model with the ORM is *relatively* simple, but some preparation is required.

SQLAlchemy provides a "DeclarativeBase" class.<br>
This class offers functionality to define columns "in a declarative manner".<br>
This means specifying columns works similar to a Python dataclass.

Just like SQLAlchemy Core, the ORM side of things also uses Metadata.<br>
It is common to first create a Base class can hold metadata with regards to Catalog/Database, Schema, and *certain* Table information.<br>
For example, when MicrosoftSQL Server is used, this also allows a single connection to be used for multiple databases.


**DeclarativeBase(SQLAlchemy)**<br>
This is the class that enabled ORM-model syntax.

**Base(Your Project, inherits from DeclarativeBase)**<br>
Holds metadata which ties specific model classes together.<br>
When using multiple databases, there might be multiple "Base" classes.

**MyModel(Your Project, inherits from Base)**<br>
This is where the table name and columns are actually defined.<br>
By inheriting from "Base", it can easilty be tied to a specific database.

**Note:** SQLAlchemy has a ``column`` function and a ``Column`` class.
Column definitions in the ORM use the class variant.
Using the function variant isn't *harmful* (it just errors-out), but it's good to know it's a common typo.

```python
import sqlalchemy as sa
from sqlalchemy import orm

class Base(orm.DeclarativeBase):
    """Base class, needs no further values for simple setups."""
    pass

class AdvancedBase(orm.DeclarativeBase):
    metadata = sa.MetaData(schema='[Database].[dbo]')
    
class MyClass(Base):
    __tablename__ = 'my_table'
    name = sa.Column(sa.VARCHAR(255))  # Will use the column "name" in SQL.
```
Each column is assigned like a class variable would be.<br>
The column name may be specified and, if omitted, will default to the name of the class variable it's assigned to.<br>
The Metaclass (or effectively the DeclarativeBase) performs the transformations to make this work.

SQLAlchemy tries not to set defaults, so anything implied/improvised by the database itself will take effect.<br>
This can create uncertainty about default value, nullable values, and database cascades.

In [4]:
# Create a base class from which all model implementations will inherit.
# This also allows child classes to be 'registered', letting them to be discovered for migrations more easily.
class Base(orm.DeclarativeBase):
    pass

In [5]:
class Customer(Base):
    __tablename__ = 'customers'
    id = sa.Column('id', sa.Integer, primary_key=True, autoincrement=True)
    name = sa.Column('name', sa.VARCHAR(255), default=None, nullable=True)

    def __repr__(self):
        return f'{type(self).__name__}(id={self.id}, name={self.name})'

Base.metadata.create_all(engine)

-----
SQLAlchemy uses a Session system to manage its ORM features.<br>
The session is layered on top of the regular SQLAlchemy connection.<br>
When using ORM objects, make sure to use the ``Session.execute`` and NOT ``Connection.execute``.

# Adding data
The session system adds a little bit of extra functionality.<br>
When creating an instance of an ORM model, it can be *added* to a session.<br>

When an object is added to a session, it is tracked as 'new'.<br>
When code leaves the associated "with" statement, the session will try to save the new object to the database.

Data can also be added in a way similar to Core functionality, further explained in the "ORM Create" notebook.

In [6]:
with orm.Session(con) as s, s.begin():
    new_customer = Customer(name='jack')
    s.add(new_customer)

In [7]:
# Read the data, similar to table queries (but not exactly)
with orm.Session(con) as s, s.begin():
    result =  s.execute(sa.select(Customer))
    for row in result:
        print(row[0])

Customer(id=1, name=jack)


-----
The select query above can be a bit confusing.<br>
In order to explain what's going on, it's a good idea to start with a bit of backgrounds.

``sa.select(*columns) -> sa.Select`` provides the basic "select" statement.<br>
Although an ORM model consists of multiple fields, it is treated as one field by the Session system.

``Session.execute(query) -> Result`` takes the ORM based query and returns a custom iterator.

``for row in results:`` tries to iterate over the result object.<br>
By default, it will try to return tuples.<br>
The number of items in ``sa.select(...)`` should always match the size of the tuple in the result.

## Cleaner Query

When iterating over the rows, the code keeps using the first column by its numeric index.<br>
This isn't very clean.<br>
SQL has something called a "scalar", which means "the first column".<br>
SQLAlchemy has some syntax sugar for this on the Result object.

``Result.scalar()`` first column of the first row (additional rows cannot be read afterwards)

``Result.scalars()`` first column of all rows.

When expecting multiple rows, just pick ``Result.scalars()``.

**Technical Note:** Technically speaking, it's not a scalar. One ORM object consists of multiple columns.<br>
However, ``sa.select()`` only got one argument. 


In [8]:
with orm.Session(con) as s, s.begin():
    result =  s.execute(sa.select(Customer))
    for customer in result.scalars():
        print(customer)

Customer(id=1, name=jack)


# Sessions
The previous examples glossed over the Session for a reason.<br>
The session can be rather vague about *when* it is saving data, or rolling back.

The following code keeps changing the name of an existing customer.<br>
The thing to look for is a chunk of logging that both has an 'UPDATE' and a 'COMMIT'.

SQLAlchemy recommends using a `sqlalchemy.orm.sessionmaker`.<br>
This is ignored for now.

The deep indentention exists for demonstration purposes.


In [79]:
name_change = {'Dave': 'Jack', 'Jack': 'Dave'}

print('Connection > Session\n')
with logs():
    with orm.Session(con) as s:
        customer = s.scalar(sa.select(Customer))
        print('Hello', customer.name)
        customer.name = name_change[customer.name]
        print('leaving session')

print('--\n\nConnection > Session > SessionTransaction\n')
with logs():
    with orm.Session(con) as s:
        with s.begin() as session_transaction:
            customer = s.scalar(sa.select(Customer))
            print('Hello', customer.name)
            customer.name = name_change[customer.name]
            print('leaving session-transaction')
        print('leaving session')

print('--\n\nConnection > Transaction > Session\n')
with logs():
    with con.begin() as transaction:
        with orm.Session(con) as s:
            customer = s.scalar(sa.select(Customer))
            print('Hello', customer.name)
            customer.name = name_change[customer.name]
            print('leaving session')
        print('leaving transaction')

print('--\n\nConnection> Transaction > Session > SessionTransaction\n')
with logs():
    with con.begin() as transaction:
        with orm.Session(con) as s:
            with s.begin() as session_transaction:
                customer = s.scalar(sa.select(Customer))
                print('Hello', customer.name)
                customer.name = name_change[customer.name]
                print('leaving session-transaction')
            print('leaving session')
        print('leaving transaction')


Connection > Session

BEGIN (implicit)
SELECT customers.id, customers.name 
FROM customers
[cached since 8229s ago] ()
Hello Dave
leaving session
ROLLBACK
--

Connection > Session > SessionTransaction

BEGIN (implicit)
SELECT customers.id, customers.name 
FROM customers
[cached since 8229s ago] ()
Hello Dave
leaving session-transaction
UPDATE customers SET name=? WHERE customers.id = ?
[cached since 8134s ago] ('Jack', 1)
COMMIT
leaving session
--

Connection > Transaction > Session

BEGIN (implicit)
SELECT customers.id, customers.name 
FROM customers
[cached since 8229s ago] ()
Hello Jack
leaving session
leaving transaction
COMMIT
--

Connection> Transaction > Session > SessionTransaction

BEGIN (implicit)
SELECT customers.id, customers.name 
FROM customers
[cached since 8229s ago] ()
Hello Jack
leaving session-transaction
UPDATE customers SET name=? WHERE customers.id = ?
[cached since 8134s ago] ('Dave', 1)
leaving session
leaving transaction
COMMIT


-----
In conclusion, starting a SessionTransaction (by using `session.begin()`) is the only surefire way to get data committed properly.<br>
Although not extremely clean, this can be setup in one line:
```python
with con.transaction(), Session(con) as session, session.begin():
    ...
```

If a SessionTransaction cannot be started in this manner, calling `session.commit()` will execute the necessary update queries.<br>
These updates still have to be comitted by using either `connection.commit()` or automatically through the transaction from `connection.begin()`.



### Instance member vs Class member
SQLAlchemy utilizes the feature of class members.

Using the `Customer` model above as an example, accessing the `.name` on an instance will give the expected value (a string).

When accessing it via the class, as `Customer.name`, it returns an instance of `InstrumentedAttribute`.

That `InstrumentedAttribute` can be used to build a query.
When used in expressions (like `Customer.id < 5`) it returns an SQL-compatible Expression.
This is how access via class is used to create parts of a query.
This is further explained in the section about 'Select'.

In [14]:
with logs(), orm.Session(con) as s:
    result =  s.execute(sa.select(Customer))
    for customer in result.scalars():
        print(customer)

BEGIN (implicit)
SELECT customers.id, customers.name 
FROM customers
[cached since 30.55s ago] ()
Customer(id=1, name=jack)
ROLLBACK


### Default Values
(onupdate, server_default, default)


In [81]:
class Product(Base):
    __tablename__ = 'products'
    id = sa.Column('id', sa.Integer, primary_key=True, autoincrement=True)
    name = sa.Column('name', sa.VARCHAR(255), default=None, nullable=True)
    price = sa.Column('price', sa.Double)
    deleted = sa.Column('deleted', sa.Boolean, default=False)
    created = sa.Column('created', sa.DateTime, server_default=sa.text('current_timestamp'))  # SQLite specific
    updated = sa.Column('updated', sa.DateTime, onupdate=sa.func.now())  # Application Logic


In [8]:
con = engine.connect()
Base.metadata.create_all(con)

In [15]:
for row in con.execute(sa.select(Customer)):
    print(row)
print(con.in_transaction())

(1, 'jack')
(2, 'john')
(3, 'jack')
(4, 'john')
True


In [ ]:
if con.in_transaction():
    con.commit()
print(con.in_transaction())
print('dialect', con.dialect.name)

## Insert
Before diving into `sqlalchemy.select(...)` statements, there has to be data added to the database.

**Note:** It is desirable to explicitly commit (in general).

For the sake of this notebook, the automatic rollback gets used everywhere.
This way SQL queries can be shown (via `with logs()`) without completely tainting the database.

In [16]:
with orm.Session(con) as s:
    obj = sa.insert(Customer).values([dict(name='jack'), dict(name='john')])
    s.execute(obj)
    # s.commit()
with orm.Session(con) as s:
    for row in s.execute(sa.select(Customer)):
        print(row)

(<__main__.Customer object at 0x0000023049891130>,)
(<__main__.Customer object at 0x0000023049891160>,)
(<__main__.Customer object at 0x0000023049891190>,)
(<__main__.Customer object at 0x00000230498911C0>,)
(<__main__.Customer object at 0x00000230498911F0>,)
(<__main__.Customer object at 0x0000023049891220>,)


### Using Instances (and object-lifetime / expire_on_commit)
Most ORMs are centered around instances, and SQLAlchemy offers simular features.

When an instance is initiallized manually, it is considered a new instance.

Instances do not have a ``.save()`` method, and instead SQLAlchemy provides a ``session.add(instance)`` system.

**However:** When the session has been comitted, the instances are considered 'expired'.
That means further use of the instances will cause a 'refetch' of data from the database.

In [ ]:
with logs(), orm.Session(con) as session:
    customer = Customer(name='Jack')
    apple = Product(name='Apple')
    pear = Product(name='Pear')
    print(apple.id, apple)
    session.add(customer)
    session.add(apple)
    session.add(pear)
    session.commit()
    # **Note:** Turn on logging to see the queries fired between print statements.
    print('--- divider, queries below got executed on demand ---')
    print('>>', customer.name)
    print('>>', apple.name)
    print('>>', pear.name)

The alternative is turning off 'expire_on_commit'.
The idea is that the objects are "close enough", that they don't need refetching.
Do note that the ``flush()/commit ()`` does not optimize the query, and is inserting in multiple steps so the ID does get returned.

In [82]:
with logs():
    # Toggle expire_on_commit to see how it will error-out.
    with orm.Session(con, expire_on_commit=False) as session:
        customer = Customer(name='Jack')
        apple = Product(name='Apple')
        pear = Product(name='Pear')
        session.add_all([customer, apple, pear])
        session.flush()
        session.commit()

    print('--- out of session scope---')
    print(apple.name)
    print(apple.id, apple.name)
    print(pear.id, pear.name)

BEGIN (implicit)
INSERT INTO customers (name) VALUES (?)
[cached since 9603s ago] ('Jack',)
INSERT INTO products (name, price, deleted, updated) VALUES (?, ?, ?, ?) RETURNING id, created
[generated in 0.00030s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Apple', None, 0, None)
ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: products
[SQL: INSERT INTO products (name, price, deleted, updated) VALUES (?, ?, ?, ?) RETURNING id, created]
[parameters: ('Apple', None, 0, None)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Bulk Insert

Should performance become a bottleneck, there are two alternatives.
1. Use dictionaries to insert data `sa.insert(Product).values([...]) -> Result`, which can still return a row count.
2. Use `bulk_save_objects(...) -> None`, this is conventient for objects but returns no row count

Understand that both methods will not update their input data/objects with the primary keys of the created objects*.

\* Postgres is the exception to this, but it is recommended to upgrade to SQLAlchemy 2.0 for this to be fully supported.
[TODO: look at insert-many-values](https://docs.sqlalchemy.org/en/20/core/connections.html#insert-many-values-behavior-for-insert-statements)

In [80]:
with logs(), orm.Session(con) as session:
    customer = Customer(name='Jack')
    apple = Product(name='Apple')
    pear = Product(name='Pear')
    session.add(customer)
    session.bulk_save_objects([apple, pear])

NameError: name 'Product' is not defined

In [ ]:
with logs(), orm.Session(con) as s:
    stmt = sa.insert(Product).values([dict(name='Carrot'), dict(name='Potato')])
    result = s.execute(stmt)
    print(result.rowcount)

## Select
With data in the tables, it's time to go on (and on) about selecting data.
SQLAlchemy tries to be consistent in its querying API, but the two approaches (ORM based and table-column based) are at odds.

When calling ``.execute``, a result object is returned.

This result is often iterable and each item returns a `Row` object (`sqlalchemy.engine.row.Row`).
This object can automatically be 'unpacked' like a tuple.

The size of this tuple is equal to the number of *positional* arguments put in `select(...)`.
That also means that a singular column gets put in a very odd tuple.


In [ ]:
# Row approach
with orm.Session(con) as s:
    result = s.execute(sa.select(Product.id, Product.name))
    for row  in result:
        print(type(row), row[0], row[1])

In [ ]:
# Two Columns, selecting ID and name (unpacking it like a tuple)
with orm.Session(con) as s:
    result = s.execute(sa.select(Product.id, Product.name))
    for product_id, product_name  in result:
        print(product_id, product_name)

It is also possible to get the values from the row like a dictionary.

This makes them accesible by their field names, or by using the class member corresponding to the query.

This only requires calling `.mappings` on the result object.

In [ ]:
# Two Columns, selecting ID and name using RowMappigns.
with orm.Session(con) as s:
    result = s.execute(sa.select(Product.id, Product.name))
    for entry in result.mappings():
        print(type(entry), entry[Product.name], entry['name'])

In [ ]:
# One Column
with orm.Session(con) as s:
    result = s.execute(sa.select(Product.name))
    for product_name, in result:  # <-- Extremely ugly trailing comma after 'product_name'
        print(product_name)

### Scalars
The final example in the cell above shows an extremely ugly trailing comma.
That trailing comma can be removed by using scalar(s).

Using a scalar effectively means only the first column will be used.

**Note:** these transformations are only applied *after* fetching data.
Even the singular `.scalar` does not limit how much data the server will return.
Although scalars are available for both the Session object and the Result object,
both will still query for all the data.

| Method | Meaning |
|--------|---------|
| `.scalars()` | First column of every row |
| `.scalar()` | First column of first row\* |

\* This still consumes the entire result.

In [ ]:
with orm.Session(con) as s:
    result = s.execute(sa.select(Product.name))
    for product_name in result.scalars():  # And the trailing comma is now gone.
        print(product_name)

In [ ]:
# .scalar() on result
with logs(logging.DEBUG), orm.Session(con) as s:
    result = s.execute(sa.select(Product.id, Product.name))
    print(result.scalar())

In [ ]:
# The scalar is also available on the session, but it STILL executes the entire query.
# You can see the data being returned in the (debug level) logs below.
with logs(logging.DEBUG), orm.Session(con) as s:
    query = sa.select(Product.id, Product.name)
    result = s.scalar(sa.select(Product.id, Product.name))
    print(result)

### ORM Scalars 
The ORM acts a little bit different when it comes to scalars.

**TLDR:** The model class is the only field in the select statement, treat it like a regular field.
That means it's safe to use ``scalars()`` to prevent the awkward tuple situation.

In [ ]:
with orm.Session(con) as s:
    result = s.execute(sa.select(Product))
    for item in result.scalars():
        print(type(item).__name__, item.id, item.name)
    

### Where / Filter
The SQL 'where' is available asboth `.where(...)` and `.filter(...)` (which is just an alias).

The way SQLAlchemy expresses filtering on columns is by using the class member that represents it.

**Note:** When interacting with boolean fields, it is necessary to use the equals symbol rather than 'is'.
The class member cannot override an 'is' operation to provide the proper transformation.

In [ ]:
with orm.Session(engine) as session:
    query = sa.select(Product).where(Product.name == 'Apple' )
    for entry in session.execute(query).scalars():
        print(entry.id, entry.name, entry.deleted)


----------
There are a lot of ways these expressions can be combined.
1. The `.where` statement will automatically apply 'AND' to its positional arguments.
2. Usage of the ampersand (a bitwise AND) will create a "BinaryExpression" which can be applied.
3. Usage of `sa.and_` effectively accompliushes the same result.
   

In [ ]:
apples_only = Product.name == 'Apple'
not_deleted = Product.deleted == False  # Don't use "is" for boolean fields.
print('Apples:', str(apples_only))
print('Available:', str(not_deleted))
print('---')

method_2 = apples_only & not_deleted
method_3 = sa.and_(apples_only, not_deleted) # This function accepts many more positional arguments.

print('1:', str(sa.select(Product).where(apples_only, not_deleted)).replace('\n', ''))
print('2:', str(sa.select(Product).where(method_2)).replace('\n', ''))
print('3:', str(sa.select(Product).where(method_3)).replace('\n', ''))

----------
Expression can also be inverted. This can be useful to recycle specific expressions.

In [ ]:
apples_only = Product.name == 'Apple'
print('A', str(apples_only))
print('B', str(~apples_only))

In [ ]:
expr = Product.name.in_(('Apple', 'Pear'))
print('Some expressions do not convert to strings very well. >>', expr)
with logs(), orm.Session(engine) as s:
    for item in s.execute(sa.select(Product).where(expr)).scalars():
        print(item.name)

### Being clever with (basic) Statements (code reuse)
Many SQLAlchemy queries can be written without a session or engine attached to it.
The session and engine only get involved when execution becomes interesting.

The query building blocks are lightweight, and could easily be reused.
Each transformation ends up returning a completely new object, meaning the old object remains untouched.
That old object usually gets discarded, but it could be recycled as a building block on its own.
This can be useful in teams to keep queries consistent.

In [ ]:
# Building Block
# Query for products which have not been marked as 'deleted'.
products = sa.select(Product).where(Product.deleted == False)
print('GET products:')
print(str(products))
with orm.Session(engine) as session:
    for entry in session.execute(products).scalars():
        print(entry.id)
# Recycling the where clause and flipping it, to remove the products which weren't part of the regular overview.
print(sa.delete(Product).where(~products.whereclause))

### Distinct

In [ ]:
with logs(), orm.Session(engine) as s:
    result = s.execute(sa.select(Product.name).distinct()).scalars()
    print(list(result))

### Result Assertions (one_or_none)
The Result object can also be used to provide some semantics.

For example, the Result object has a `one()` method which ensures that only one entry is returned.
That means it will raise an error when no records are found, or multiple records are found.

These errors are `sqlalchemy.exc.NoResultFound` and `sqlalchemy.exc.MultipleResultsFound` respectively.

- `one() -> Row`
- `one_or_none() -> Optional[Row]`
- `scalar_one() -> T`, equivalent to `.scalars().one()`
- `scalar_one_or_none() -> Optional[T]`, equivalent to `.scalar().one_or_none()`
- `first()` -> `one_or_none()`

In [ ]:
with orm.Session(engine) as s:
    result = s.execute(sa.select(Product).where(Product.name == 'Apple'))
    value = result.one()
    print(value.name)

### Order By
Just like regular SQL, rows can be sorted by specific columns.

In [ ]:
query = (
    sa.select(Product.id)
    .order_by(Product.id.desc(), Product.name.asc()) 
)
print(str(query))

### Group By

In [ ]:
with orm.Session(con) as s:
    # Inserting an extra apple so 'group by' actually does something.
    s.execute(sa.insert(Product).values(name='Apple'))
    query = sa.select(Product.name, sa.func.count(Product.name)).group_by(Product.name)
    for item in s.execute(query).mappings():
        print(item)


In [ ]:
with orm.Session(con) as s:
    # Inserting an extra apple so 'group by' actually does something.
    s.execute(sa.insert(Product).values(name='Apple'))
    count_column = sa.func.count(Product.name).label('Total_Products')
    query = sa.select(Product.name, count_column).group_by(Product.name)
    for item in s.execute(query).mappings():
        # It can still be accessed as long as it's the right calculated column.
        print(item[Product.name], item[count_column])

In [ ]:
with orm.Session(engine) as s:
    query = sa.select(Product)
    # it = session.execute(query.with_only_columns(sa.func.count(Product.id).label('count')))
    it = s.execute(sa.select(sa.func.count(Product.id).label('count')))
    print(it.scalar())

### Calculated Columns

In [ ]:
with logs(), orm.Session(engine) as s:
    result = s.execute(sa.select(Product.name +' = '+ Product.name))
    for item in result.mappings():
        print(item)

### Defer Columns (Lazy Loaded Columns)
It's also possible to completely omit columns from the query, even if they're part of the model.

This is usually called 'deferred loading'.
Things like deferred loading come in useful to steer the ORM.
Database columns that are defined as TEXT in SQL can be slow when there are a lot of them.
When there is a relationship between two SQLAlchemy models, it can prevent eager loading.

Select.only_columns

Select.with_only_columns

Select.reduce_columns

In [ ]:
with logs(), orm.Session(engine) as s:
    query = (
        sa.select(Product)
        .options(sa.orm.defer(Product.name, raiseload=True))
    )
    query.reduce_columns
    for item in s.execute(query).scalars():
        print(item.id)
        try:
            print(item.name)
        except sa.exc.InvalidRequestError as ex:
            print('(blocked lazy load)', str(ex))

## Update

In [ ]:
q =  sa.update(Product).where(Product.name == 'Apple')

In [ ]:
q.values(

In [ ]:
with orm.Session(con) as s:
    soft_delete_apples = sa.update(Product).where(Product.name == 'Apple').values({Product.deleted: True})
    result = s.execute(soft_delete_apples)
    print(result.rowcount, 'rows soft-deleted')
    
    undelete_all = sa.update(Product).where(Product.deleted == True).values({'deleted': False}) 
    result = s.execute(undelete_all)
    print(result.rowcount, 'rows affected')

## Delete


In [ ]:
with orm.Session(con) as s:
    s.execute(sa.delete(Customer))
    s.execute(sa.delete(Product))
    s.commit()

# Compatibility with Core
The dunder table (``__table__``) is available in case a function *only* accepts a table.<br>
Using `sa.select(Model)` will select all the fields, just like `sa.select(Table)` would.<br>
It works the same for columns, but no longer needing the `.c` shorthand in-between.


In [87]:
for row in con.execute(sa.select(Customer.id)):
    print(row)

(1,)
